In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

from collections import OrderedDict

In [2]:
tr_batchsize = 32
val_test_batchsize = 16
epochs = 60
lr = 0.00005

In [3]:
# By defalt, set device to the CPU
deviceFlag = torch.device('cpu')

# Default is CPU, but as long as GPU is avaliable, then use GPU
if torch.cuda.is_available():
    print(f'Found {torch.cuda.device_count()} GPUs.')
    deviceFlag = torch.device('cuda:0') # Manually pick your cuda device. By default is 'cuda:0'

print(f'Now the deivce is set to {deviceFlag}')

Found 1 GPUs.
Now the deivce is set to cuda:0


# Data Loading and Transformations

In [4]:
training_transforms = transforms.Compose([
    # Randomly rotate it 90 degrees
    transforms.RandomRotation(90),
    # Randomly crop an area of the flower of size 224x224
    transforms.RandomResizedCrop(224),
    # Flip it horizontally, or don't
    transforms.RandomHorizontalFlip(),
    # Flip it vertically, or don't
    transforms.RandomVerticalFlip(),
    # Convert the image to a Tensor
    transforms.ToTensor(),
    # Normalize the Tensor values so that they're easier for the
    # model to train from
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

testing_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

# Load the datasets of the Flower102 images
train_dataset = datasets.Flowers102(root = './dataset', split = 'train', transform = training_transforms, download = True)
valid_dataset = datasets.Flowers102(root = './dataset', split = 'val', transform = validation_transforms, download = True)
test_dataset = datasets.Flowers102(root = './dataset', split = 'test', transform = testing_transforms, download = True)


# Create the loaders for the datasets, to be used to train, validate and test the model
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = tr_batchsize,
                                           shuffle = True)

validate_loader = torch.utils.data.DataLoader(dataset = valid_dataset,
                                           batch_size = val_test_batchsize)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = val_test_batchsize)

In [5]:
import model_flower

In [6]:
model = model_flower.FlowerModel()
model.to(deviceFlag)
model

FlowerModel(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU()
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
   

In [7]:
# for params in model.parameters():
#     params.requries_grad = False

# Define Loss Function and Optimizer

In [8]:
# Negative Log Likelihood Loss
# criterion = nn.NLLLoss()

# Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# optimizer 1
optimizer = optim.Adam(model.parameters(), lr = lr)

# optimizer 2
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay = 0.005, momentum = 0.9)

In [9]:
import model_train
import model_test

In [10]:
model_train.train_classifier(model, train_loader, validate_loader, optimizer, criterion,
                             device_flag=deviceFlag, epochs=epochs,
                             validate_steps=100, validate_stepped=True, validate_epoch=False,
                             validate_end=True)

[1/60] Epoch 1 completed on Batch 32 in 13.2895 seconds (13.2895 in total) 

[2/60] Epoch 2 completed on Batch 64 in 10.4957 seconds (23.7852 in total) 

[3/60] Epoch 3 completed on Batch 96 in 10.5799 seconds (34.3651 in total) 

[4/60] Batch: 100... Training Loss since last stepped validation: 4.2370... Validation Loss: 3.8870... Validation Accuracy: 0.0918
[4/60] Epoch 4 completed on Batch 128 in 19.3139 seconds (53.6790 in total) 

[5/60] Epoch 5 completed on Batch 160 in 10.5576 seconds (64.2366 in total) 

[6/60] Epoch 6 completed on Batch 192 in 10.5125 seconds (74.7501 in total) 

[7/60] Batch: 200... Training Loss since last stepped validation: 3.5690... Validation Loss: 3.4731... Validation Accuracy: 0.1615
[7/60] Epoch 7 completed on Batch 224 in 19.2789 seconds (94.0290 in total) 

[8/60] Epoch 8 completed on Batch 256 in 10.6297 seconds (104.6588 in total) 

[9/60] Epoch 9 completed on Batch 288 in 10.5968 seconds (115.2556 in total) 

[10/60] Batch: 300... Training Loss s

1920

In [18]:
model_test.test_accuracy(model, test_loader, device_flag=deviceFlag)

Test Accuracy: 0.3604545295238495 2216 / 6149 correctly predicted.


In [12]:
import datetime

In [13]:
torch.save(model.state_dict(), "models/" + str(datetime.datetime.now()).replace(":","-")
           + f" b{tr_batchsize}-e{epochs}-lr{lr}" "-model.pt")

In [14]:
# Stop Run All here
assert False

AssertionError: 

In [17]:
# Reload imports in the case that they are changed
from importlib import reload

# If not loaded into cache yet, import them
import model_flower
import model_train
import model_test

reload(model_flower)
reload(model_train)
reload(model_test)

<module 'model_test' from 'Z:\\Files\\University\\Exams\\INT2\\GitHub Project\\model_test.py'>

In [ ]:
# total_step = len(train_loader)

# for epoch in range(epochs):
#     for i, (images, labels) in enumerate(train_loader):  
#         # Move tensors to the configured device
#         images = images.to(deviceFlag)
#         labels = labels.to(deviceFlag)
        
#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)
        
#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, epochs, i+1, total_step, loss.item()))
            
#     # Validation
#     with torch.no_grad():
#         correct = 0
#         total = 0
#         for images, labels in validate_loader:
#             images = images.to(deviceFlag)
#             labels = labels.to(deviceFlag)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#             del images, labels, outputs
    
#         print('Accuracy of the network on the {} validation images: {} %'.format(total, 100 * correct / total)) 


In [ ]:
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.to(deviceFlag)
#         labels = labels.to(deviceFlag)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         del images, labels, outputs

#     print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))   